In [41]:
import numpy as np
import cvxpy as cp

from opftools.result import *



def runMOPF(case, genData, T=1, init_sot=0.1, robust=False, verb=False, solver=None, **kwargs):
    '''Find the solution to the OPF specified.
    The solver uses a sparse representation of the problem

    Type defines the type of relaxation used. These are 
    0 (default) -> SDR
    1 -> SOCR
    2 -> TCR
    3 -> STCR

    verb invokes the verbose option in cvxpy

    Returns:
        RunResult instance with the optimal values
        -> None if optimizer does not converge 
    '''
    # load data from case
    n = case.N
    baseMVA = case.mva
    Load_data = case.loadData
    Gen_data = case.genData
    Costs_data = case.cost
    Y = case.adj
    Sij = case.smax
    v_lim = case.vlim
    lines = case.getLines()

    renew = genData.renew
    avgs = genData.avgs
    stds = genData.stds
    batteries = genData.batteries
    diesel = genData.diesel
    Load_data = genData.Load_data
    renew_lim = genData.renew_lim

    #Voltage matrix
    V = [cp.Variable((n,n), hermitian=True) for _ in range(T)]

    # power transfer variables
    pij = cp.Variable((len(lines), T))
    pji = cp.Variable((len(lines), T))
    qij = cp.Variable((len(lines), T))
    qji = cp.Variable((len(lines), T))

    # total generation variable
    pi_g = cp.Variable((n, T))
    qi_g = cp.Variable((n, T))
    renew_real = cp.Variable((len(renew), T))
    renew_react = cp.Variable((len(renew), T))

    

    diesel_real = cp.Variable((len(diesel), T))
    diesel_react = cp.Variable((len(diesel), T))
    sot = cp.Variable((n, T))
    pow_out = cp.Variable((n, T))
    pow_in = cp.Variable((n, T))

    max_out = 20
    max_in = 20
    max_charge = 100
    eta_in = 0.95
    eta_out = 0.95
    initial_sot = np.ones(n)*0.1



    # Define constraints
    constraints=[]

    constraints += [pow_out >= 0, pow_in >= 0, sot >= 0]
    constraints += [pow_out <= max_out, pow_in <= max_in, sot <= max_charge]

    #Constraints on active and reactive generation (min-max)
    constraints += [diesel_real >= 0, diesel_real <= 50]
    constraints += [diesel_react >= -50, diesel_react <= 50]
    constraints += [renew_react >= -50, renew_react <= 50]
    constraints += [renew_real <= renew_lim]
    constraints += [renew_real >= 0]


    # Calculate the sum of all inbound power flows to each bus
    for t in range(T):

        if robust:
            constraints += [cp.norm((renew_real[:,t]-avgs[t])/stds[t]) <= 1]

        for i in range(n) :  
            psum = 0
            qsum = 0
            for line in range(len(lines)):
                start, end = lines[line]
                if start == i:
                    psum += pij[line, t]
                    qsum += qij[line, t]
                elif end == i:
                    psum += pji[line, t]
                    qsum += qji[line, t]

            # Sum pij = pi
            if t>0 and i in batteries:
                constraints += [sot[i, t] == sot[i, t-1] + (pow_in[i, t]*eta_in-pow_out[i, t]/eta_out)]
            else:
                constraints += [sot[i, t] == initial_sot]
                constraints += [pow_in[i, t]==0, pow_out[i, t] == 0]
            gensum = 0
            rgensum = 0
            if i in batteries:
                gensum += pow_in[i, t]-pow_out[i, t]
            if i in renew:
                gensum += renew_real[renew.index(i)]
                rgensum += renew_react[renew.index(i)]
            if i in diesel:
                gensum += diesel_real[diesel.index(i)]
                rgensum += diesel_react[diesel.index(i)]
            constraints += [psum == gensum-Load_data[i,0, t]]
            # Sum qij = qi 
            constraints += [qsum == rgensum-Load_data[i,1, t]]

            

            # Voltage limits
            constraints+=[cp.real(V[t][i,i])>= (v_lim[i,1])**2, cp.real(V[t][i,i]) <= (v_lim[i,0])**2]
            constraints += [cp.imag(V[t][i,i]) == 0]

        # Power flow equations (sparse representation)
        for line in range(len(lines)):
            i, j = lines[line]
                
            #Powerflow
            constraints+=[pij[line, t] + 1j*qij[line, t]==(V[t][i,i]-V[t][i,j])*np.conjugate(Y[i,j])] 
            constraints+=[pji[line, t] + 1j*qji[line, t]==(V[t][j,j]-V[t][j,i])*np.conjugate(Y[j,i])] 
            
            if not Sij[i,j] == 0:
            #Apparent power capacity S_bar
                constraints+=[cp.square(pij[line, t])+cp.square(qij[line, t])<=cp.square(Sij[i,j])]
                constraints+=[cp.square(pji[line, t])+cp.square(qji[line, t])<=cp.square(Sij[j,i])]
                
        for line in range(len(lines)):
            i, j = lines[line]
            constraints+=[cp.norm(cp.hstack([2*V[t][i,j],(V[t][i,i]-V[t][j,j])])) <= cp.real(V[t][i,i]+V[t][j,j])]
            constraints+=[cp.norm(cp.hstack([2*V[t][j,i],(V[t][j,j]-V[t][i,i])])) <= cp.real(V[t][j,j]+V[t][i,i])]

    # Define costs
    Costs = cp.sum(diesel_real)
    #for i in range(n): 
    #    c0 = Costs_data[i][2]
    #    c1 = Costs_data[i][1]
    #    c2 = Costs_data[i][0]
    #    if c1 > 0.0: # Bus has a generator installed
    #        Costs += c0+c1*pi_g[i]*baseMVA+c2*cp.square(pi_g[i]*baseMVA)
    
    prob = cp.Problem(cp.Minimize(Costs), constraints)
    prob.solve()
    ans = RunResult()
    return prob.value
    try:
        ans.setAll(pi_g.value*baseMVA, qi_g*baseMVA, V.value, prob.value)
    except TypeError:
        return None
    return ans

In [5]:
from opftools import *

In [7]:
case39 = loadCase("C:\\Users\\Jean-Luc\\Documents\\PolyMtl\\Git\\OPF_Tools\\cases\\case39.json")

In [12]:
result = runOPF(case39)

In [17]:
np.sum(result.p/case39.mva)

63.007585748329944

In [16]:
np.sum(case39.loadData[:,0])

62.542300000000004

In [18]:
np.sum(case39.genData[:,0] == 0)

29

In [32]:
diesel = [30]
batteries = [31, 32, 33, 34, 35, 36, 37, 38]
renew = [31, 32, 33, 34, 35, 36, 37, 38]

class gens:

    def __init__(self, diesel, batteries, renew, avgs, stds, Load_data, renew_lim):
        self.renew = renew
        self.diesel = diesel
        self.batteries = batteries
        self.avgs = avgs
        self.stds = stds
        self.Load_data = Load_data
        self.renew_lim = renew_lim


In [30]:
import os
pv_directory   = 'PV Generation Data'
load_directory = 'Building Load Data'
wind_directory = 'Wind Generation Data'

# ALREADY GENERATED, takes 5 minutes to generate again so keep commented out
#dp.generate_json_from_pv_data(pv_directory) 
#dp.generate_json_from_bldg_data(load_directory) 
#dp.generate_json_from_wind_data(wind_directory) 

with open(os.path.join(pv_directory, 'pv_data.json'), 'r') as json_file:
    pv_dict = json.load(json_file)

for key in pv_dict.keys(): # get data from first key only (CAPTL_WF)
    solar_data = np.array(pv_dict[key])
    break

# for key, value in pv_dict.items():
    # print(key, value['0'])
    # break

with open(os.path.join(load_directory, 'real_data.json'), 'r') as json_file:
    real_load_dict = json.load(json_file)

with open(os.path.join(load_directory, 'reactive_data.json'), 'r') as json_file:
    reactive_load_dict = json.load(json_file)

with open(os.path.join(wind_directory, 'wind_data.json'), 'r') as json_file:
    wind_dict = json.load(json_file)

real_load_dict.keys()
real_load_dict['GeiselLibrary0']['0']
def toNDct(dict):
    ans = []
    for bld in dict:
        ls2 = []
        for day in dict[bld]:
            ls2.append(dict[bld][day])
        ans.append(ls2)
    return np.asarray(ans)

real_load = toNDct(real_load_dict)
react_load = toNDct(reactive_load_dict)
wind = toNDct(wind_dict)
solar = toNDct(pv_dict)


In [33]:
Load_data = np.zeros((39, 2, 24))
Load_data[:27,:] = np.stack((real_load[:,0,0::4], react_load[:,0,0::4]), axis=1)
renew_lim = wind[:8,0,0::4]+solar[:8,0,0::4]
avgs = np.mean(wind[:8,:,0::4]+solar[:8,:,0::4], axis=1)
stds = np.std(wind[:8,:,0::4]+solar[:8,:,0::4], axis=1)

genDat = gens(diesel, batteries, renew, avgs, stds, Load_data, renew_lim)

In [ ]:
re

In [38]:
renew_lim.shape

(8, 24)

In [42]:
runMOPF(case39, genDat, T=24)

In [25]:
Load_data[1,:,:]

array([[0.49237637, 0.49226352, 0.49849627, 0.48913403, 0.49337623,
        0.49415565, 0.51326328, 0.53574447, 0.53470261, 0.53113486,
        0.53414888, 0.63149386, 0.77623002, 0.75821017, 0.76987786,
        0.75357432, 0.7513922 , 0.75420153, 0.7597034 , 0.74599399,
        0.73612788, 0.50296023, 0.49323321, 0.48795702],
       [0.75307989, 0.75774743, 0.74757947, 0.74505483, 0.74562743,
        0.73920738, 0.76247571, 0.77008433, 0.81182329, 0.81675111,
        0.80728588, 0.79207732, 0.76694371, 0.76417615, 0.76793275,
        0.75689721, 0.73359418, 0.75863236, 0.72957732, 0.70067844,
        0.69473556, 0.75221231, 0.74087313, 0.73925076]])

In [27]:
renew_real

array([[0.00199402, 0.04417514, 0.03876486, 0.17492356, 0.31108226,
        0.48871977, 0.61088906, 0.69841441, 1.08520606, 1.36829583,
        1.25955209, 1.25065741, 1.23044087, 0.90327754, 0.39972868,
        0.39386763, 0.52803717, 0.45375236, 0.6331933 , 0.78738626,
        0.56917166, 0.43842323, 0.28152513, 0.27070457],
       [0.76430181, 0.88204358, 0.82365568, 0.83717935, 0.9471933 ,
        0.95449179, 0.95320382, 1.00627193, 1.28572025, 1.56188329,
        1.50331855, 1.24462166, 1.09384438, 0.94682202, 0.81240631,
        0.50156548, 0.20736622, 0.1653808 , 0.47053773, 0.78544596,
        0.91531609, 0.6904583 , 0.70290866, 0.82140174],
       [0.71416467, 0.92373958, 0.9840418 , 0.89549499, 0.93630843,
        0.93602599, 0.92373958, 0.95283152, 0.8528451 , 1.14338488,
        1.34799135, 1.32852458, 1.03172002, 0.90254947, 0.74931378,
        0.67657746, 0.27695055, 0.20475755, 0.60302217, 0.79663889,
        0.92049146, 0.74805818, 0.73435955, 0.56658664],
       [0.302